In [1]:
from tensorflow.python.keras.datasets import cifar10
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
import random
import time
import os

Using TensorFlow backend.


In [2]:
BATCH_SIZE = 256
SAMPLES_PER_EPOCH = 50000
EPOCH_NUM = 10
DIR = 'data'
LOG_DIR = 'summaries'

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def conv_layer(input, shape):
    W = weight_variable(shape)
    variable_summaries(W)
    b = bias_variable([shape[3]])
    variable_summaries(b)
    return tf.nn.relu(conv2d(input, W) + b)


def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    variable_summaries(W)
    b = bias_variable([size])
    variable_summaries(b)
    return tf.matmul(input, W) + b
  
def get_batch(features,labels,batch_size): 
    num_images = features.shape[0]
    idx = np.random.choice(num_images,
                           size=batch_size,
                           replace=False)
    features_batch = features[idx, :, :, :]
    labels_batch = labels[idx, :]
    return features_batch, labels_batch

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


In [4]:
(train_features, train_labels), (test_features, test_labels) = cifar10.load_data()

num_train, img_rows, img_cols, img_channels =  train_features.shape
num_test =  test_features.shape[0]
num_classes = np.unique(train_labels).shape[0]

train_features = train_features.astype('float32')/np.max(train_features)
test_features = test_features.astype('float32')/np.max(test_features)


train_labels = np_utils.to_categorical(train_labels, num_classes)
test_labels = np_utils.to_categorical(test_labels, num_classes)

print(train_features.shape, train_labels.shape)

(50000, 32, 32, 3) (50000, 10)


In [5]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32 , shape=[None , img_rows,img_cols,img_channels]) 
y_ = tf.placeholder(tf.float32 , shape=[None, 10]) 
keep_prob = tf.placeholder(tf.float32)

test_features, test_labels = get_batch(test_features,test_labels,500)

In [27]:
with tf.name_scope('conv_1'): 
    conv1 = conv_layer(x , shape=[3, 3, img_channels, 64])
    conv1_pool = max_pool_2x2( conv1 )

with tf.name_scope('conv_2'): 
    conv2 = conv_layer(conv1_pool , shape=[3, 3, 64, 128])
    conv2_pool = max_pool_2x2 ( conv2 )

with tf.name_scope('conv_3'): 
    conv3 = conv_layer(conv2_pool , shape=[3, 3, 128, 256] )
    conv3_pool = max_pool_2x2 ( conv3 )
    conv3_flat = tf.contrib.layers.flatten(conv3_pool) 

with tf.name_scope('full_1'): 
    full_1 = tf.nn.relu(full_layer(conv3_flat , 512))

with tf.name_scope('dropout'): 
    full1_drop = tf.nn.dropout(full_1 , keep_prob=keep_prob) 

with tf.name_scope('activations'): 
    y_conv = full_layer(full1_drop , 10) 
    tf.summary.scalar('cross_entropy_loss',y_conv)
    variable_summaries(y_conv)

with tf.name_scope('cross'): 
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_conv , labels=y_))
    tf.summary.scalar('cross_entropy', cross_entropy )
    variable_summaries(cross_entropy)

In [28]:
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv , 1), tf.argmax(y_, 1))
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))
    tf.summary.scalar('accuracy', accuracy)

print(test_features.shape, test_labels.shape)

(500, 32, 32, 3) (500, 10)


In [29]:
# Merge all the summaries
merged = tf.summary.merge_all()

In [30]:
saver = tf.train.Saver(max_to_keep = 7, keep_checkpoint_every_n_hours = 1)

In [31]:
with tf.device("/gpu:0"):     
    with tf.Session() as sess: 
        train_writer = tf.summary.FileWriter(LOG_DIR + '/train', graph = tf.get_default_graph())
        test_writer = tf.summary.FileWriter(LOG_DIR + '/test', graph = tf.get_default_graph())

        sess.run(tf.global_variables_initializer())
        start_time = time.time()

        for j in range(EPOCH_NUM):
            for i in range(SAMPLES_PER_EPOCH // BATCH_SIZE):
                batch_trainf, batch_trainl = get_batch(train_features,train_labels,BATCH_SIZE)
                sess.run(train_step, feed_dict={x: batch_trainf, y_: batch_trainl, keep_prob: 0.5}) 
            batch_trainf, batch_trainl = get_batch(test_features,test_labels,32)
            summary, train_accuracy = sess.run([merged, accuracy], feed_dict={x:batch_trainf, y_:batch_trainl, keep_prob:1.0})
            train_writer.add_summary(summary,j)
            print("time {}, epoch {}, training accuracy {}".format(time.time() - start_time, j, train_accuracy))
            saver.save(sess, os.path.join(DIR, "model_ckpt"),global_step = j)

        summary, test_accuracy = np.mean([sess.run([merged, accuracy] , feed_dict={x:test_features, y_:test_labels, keep_prob:1.0})])
        test_writer.add_summary(summary,j)
        print("test accuracy: {}".format(test_accuracy))

InvalidArgumentError: tags and values not the same shape: [] != [32,10] (tag 'activations_3/cross_entropy_loss')
	 [[node activations_3/cross_entropy_loss (defined at d:\programs\python\python36\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

Original stack trace for 'activations_3/cross_entropy_loss':
  File "d:\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\programs\python\python36\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "d:\programs\python\python36\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "d:\programs\python\python36\lib\site-packages\tornado\platform\asyncio.p

In [ ]:
train_var = [x, y_, keep_prob, accuracy]
tf.add_to_collection('train_var', train_var[0])
tf.add_to_collection('train_var', train_var[1])
tf.add_to_collection('train_var', train_var[2])
tf.add_to_collection('train_var', train_var[3])

saver.export_meta_graph(os.path.join(DIR,"model_ckpt-9.meta"), collection_list = ['train_var'])

In [ ]:
tf.reset_default_graph()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    start_time = time.time()
    saver = tf.train.import_meta_graph(os.path.join(DIR,"model_ckpt-9.meta"))
    saver.restore(sess,os.path.join(DIR,"model_ckpt-9"))
    x = tf.get_collection('train_var')[0]
    y_ = tf.get_collection('train_var')[1]
    keep_prob = tf.get_collection('train_var')[2]
    accuracy = tf.get_collection('train_var')[3]
    
 
    test_accuracy = np.mean([sess.run(accuracy , feed_dict={x:test_features, y_:test_labels, keep_prob:1.0})])
    print("test accuracy: {}".format(test_accuracy))